In [14]:
import scipp as sc
import plopp as pp
import scippneutron as scn
import scippnexus as snx
import h5py
from pathlib import Path
import numpy as np
from easyscience.Objects.new_variable import Parameter
from easyscience.Objects.ObjectClasses import BaseObj
from easyscience.fitting import Fitter

In [15]:
datafile = "20240914/BIFROST_20240914T053723.h5"

In [16]:
from ess.spectroscopy.indirect import bifrost
from bifrost2409.config import POOCH_DATA_DIR, INTERIM_DATA_DIR


In [17]:
targets = ['energy_momentum_events']
target_files = {target: INTERIM_DATA_DIR / f'{Path(datafile).stem}_{target}.h5' for target in targets}
if all(file.exists() for file in target_files.values()):
    from scipp.io import load_hdf5
    objects = {target: load_hdf5(file) for target, file in target_files.items()}
else:
    data = bifrost(POOCH_DATA_DIR / datafile, is_simulated=True)
    objects = {target: data[target] for target in targets}
    for target in targets:
        objects[target].save_hdf5(target_files[target])


In [18]:
energy_momentum_events = objects['energy_momentum_events']

In [19]:
def hist_E_plane(events, q_x_range, q_bins, e_bins):
    a = events.bin(table_momentum_x=q_x_range)
    # Remove coordinates and event coordinates that we're not using:
    for coord in ('a3', 'a4', 'detector_number', 'final_energy'):
        del a.coords[coord]
    for coord in ('event_time_offset', 'event_time_zero', 'frame_time', 'incident_energy', 'lab_momentum_x', 'lab_momentum_z'):
        del a.bins.coords[coord]
    # drop the non-energy_transfer dimensions before binning in Q
    for dim in ('setting', 'event_id'):
        a = a.bins.concat(dim)
    return a.bin(energy_transfer=e_bins, table_momentum_z=q_bins).hist()['table_momentum_x', 0]
 

In [20]:
astar = 2 * np.pi / 6.56162

In [ ]:
my_events=hist_E_plane(energy_momentum_events, sc.array(values=[2 * astar - 0.2,  2 * astar + 0.2], dims=['table_momentum_x'], unit='1/angstrom'), 200, 50)
my_events.plot(norm='log')

In [ ]:
my_events

In [23]:
def prepare_data_for_cut(events):
    q_x_range=sc.array(values=[2 * astar - 0.2,  2 * astar + 0.2], dims=['table_momentum_x'], unit='1/angstrom')
    q_z_range=sc.array(values=[-2 * astar,  2 * astar], dims=['table_momentum_z'], unit='1/angstrom')
    a = events.bin(table_momentum_x=q_x_range,table_momentum_z=q_z_range)
    # Remove coordinates and event coordinates that we're not using:
    for coord in ('a3', 'a4', 'detector_number', 'final_energy'):
        del a.coords[coord]
    for coord in ('event_time_offset', 'event_time_zero', 'frame_time', 'incident_energy', 'lab_momentum_x', 'lab_momentum_z'):
        del a.bins.coords[coord]
    # drop the non-energy_transfer dimensions before binning in Q
    for dim in ('setting', 'event_id'):
        a = a.bins.concat(dim)    
    return a
    
prepared_data=prepare_data_for_cut(energy_momentum_events)

In [24]:
# q_z_value=1.0
# q_z_width=0.1
# q_z_range=sc.array(values=[q_z_value - q_z_width/2, q_z_value + q_z_width/2], dims=['table_momentum_z'], unit='1/angstrom')
# q_z_range

# b = a.bin(table_momentum_z=q_z_range)
# b

In [ ]:
def const_Q_cut(events, q_z_value,q_z_width,e_bins):
    q_z_range=sc.array(values=[q_z_value - q_z_width/2, q_z_value + q_z_width/2], dims=['table_momentum_z'], unit='1/angstrom')
    a = events.bin(table_momentum_z=q_z_range)
    a=a.bin(energy_transfer=e_bins).hist()

    a = a.assign_coords(energy_transfer=sc.midpoints(a.coords['energy_transfer']))

    a=a['table_momentum_x', 0]['table_momentum_z', 0]    
    a.variances = a.values+1.0
    return a


mycut=const_Q_cut(prepared_data,1.0,0.1,201)

mycut.plot(norm='log')

# mycut

In [ ]:

import matplotlib.pyplot as plt
# Function to calculate the Gaussian + background model for fitting
def gaussian_model(E: np.ndarray) -> np.ndarray:

    y= A.value * np.exp(-((E - E0.value) ** 2) / (2 * sigma.value ** 2)) + B.value
    # y_has_nans = np.any(np.isnan(y))
    # print(f"y has NaNs: {y_has_nans}")
    return y



# Define the Q range for constant-Q cuts and width
h_cut_centers = np.linspace(0.1, 1.6, 21)  # Centers of the Q cuts
# h_cut_centers=[1.0]
h_cut_width = 0.1  # Width of Q cuts
h_cut_width = (1.6-0.1)/21  # Width of Q cuts

# Store the fit results for each Q cut
fit_results = []

e_bins=201
e_min=0.1
# Loop over each Q cut
for h_center in h_cut_centers:
    print(h_center)
    this_data=const_Q_cut(prepared_data,h_center,h_cut_width,e_bins)
    E_values=this_data.coords['energy_transfer'].values
    intensity_cut=this_data.values
    intensity_error_cut=np.sqrt(this_data.variances)

    indices=E_values>e_min
    E_values=E_values[indices]
    intensity_cut=intensity_cut[indices]
    intensity_error_cut=intensity_error_cut[indices]
        
    
    # Define the Gaussian parameters to fit (A = amplitude, E0 = center, sigma = width, B = background)
    A = Parameter(name='A', value=np.max(intensity_cut), fixed=False,min=0)
    E0 = Parameter(name='E0', value=E_values[np.argmax(intensity_cut)], fixed=False)
    sigma = Parameter(name='sigma', value=0.05, fixed=False,min=0)
    B = Parameter(name='B', value=np.min(intensity_cut), fixed=False)
    
    # Create the base object for the Gaussian
    gaussian = BaseObj(name='gaussian', A=A, E0=E0, sigma=sigma, B=B)
    # simple_gaussian = BaseObj(name='simple_gaussian', A=A)
    
    # Create the Fitter object
    fitter = Fitter(gaussian, gaussian_model)
    # fitter = Fitter(simple_gaussian, gaussian_model)
    
    # Fit the data for this cut
    res = fitter.fit(x=E_values, y=intensity_cut,weights=1/intensity_error_cut)
    
    # Save the fit parameters for this cut
    fit_results.append({
        'h_center': h_center,
        'A': A,
        'E0': E0,
        'sigma': sigma,
        'B': B,
        'intensity_cut': intensity_cut,
        'intensity_error_cut': intensity_error_cut,
        'E_values': E_values
    })

# Show the fit results
for result in fit_results:
    print(f"h_center: {round(result['h_center'], 2)}, {result['A']}, {result['E0']}, {result['sigma']}, {result['B']}")

plt.figure(figsize=(15, 20))
for i, result in enumerate(fit_results):
# for i in range(1):

    h_center = result['h_center']
    A = result['A']
    E0 = result['E0']
    sigma = result['sigma']
    B = result['B']
    intensity_cut = result['intensity_cut']
    intensity_error_cut = result['intensity_error_cut']
    E_values = result['E_values']
    
    plt.subplot(7, 3, i + 1)
    plt.errorbar(E_values, intensity_cut, intensity_error_cut, label='Data')
    plt.plot(E_values, gaussian_model(E_values), label='Fit')
    plt.title(f'h_center = {h_center:.2f}')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:

h_values =  np.array([result['h_center'] for result in fit_results])  
E0_values = np.array([result['E0'].value for result in fit_results]) 
E0_errors = np.array([result['E0'].error for result in fit_results]) 
A_values = np.array([result['A'].value for result in fit_results]) 
A_errors = np.array([result['A'].error for result in fit_results]) 

# Define the parameters to fit (J, S)
K = Parameter(name='K', value=1.0, fixed=False,min=0)  

# def dispersion_model(h: np.ndarray) -> np.ndarray:
#     return 4 * J.value * S * (1 - np.cos(np.pi*  h))+D.value

def dispersion_model(h: np.ndarray) -> np.ndarray:
    astar = 2 * np.pi / 6.56162
    return K.value*np.abs(np.sin(np.pi*h/2/astar))
    return 4 * J.value * S * (1 - np.cos(np.pi*  h))+D.value

    

# Create a BaseObj and fit the data
phonon_dispersion = BaseObj(name='phonon_dispersion', K=K)
fitter = Fitter(phonon_dispersion, dispersion_model)
res = fitter.fit(x=h_values, y=E0_values, weights=1/E0_errors)

# Extract the fitted parameters
fitted_K = K

# Print the fitted parameters
print(f"Fitted K: {fitted_K}")

# Optional: Plot the fit against the data
import matplotlib.pyplot as plt

# Generate model values using the fitted parameters
h_fit = np.linspace(min(h_values), max(h_values), 100)
E_fit = dispersion_model(h_fit)

# Plot the data and the fitted model
plt.errorbar(h_values/astar, E0_values, yerr=E0_errors, fmt='o', label='Data')
plt.plot(h_fit/astar, E_fit, label='Fitted Dispersion', color='red')
plt.xlabel('(h,0,0) (R.L.U.)')
plt.ylabel('E (meV)')
plt.title('Fitted phonon dispersion')
plt.legend()
plt.ylim(0,1.6)
plt.show()


# # Plot the data and the fitted model
# plt.errorbar(h_values, A_values, yerr=A_errors, fmt='o', label='Data')
# plt.xlabel('(h,0,0) (R.L.U.)')
# plt.ylabel('Intensity')
# # plt.title('Fitted Ferromagnetic Dispersion Relation')
# plt.legend()
# plt.show()
